In [65]:
import pandas as pd
import numpy as np
import networkx as nx
import os
from os import listdir
import random

# Klebsiella pneumoniae subsp. pneumoniae MGH 78578

In order to have the same RefSeq on both databases, this subspecie is used

In [107]:
bacteria = 'Klebsiella'
root = 'E:/User/bruna.fistarol/Documents/GitHub'

# Data From PATRIC

## Genomic Features

The table below contains a list of genomic features, including coding DNA.

Each feature is solely identified by BRC ID and associated to a protein family referred as PATRIC genus-specific families (PLfams). Most of the genes has the associated RefSeq.

In [67]:
features = pd.read_csv('genome_features.csv')

Through this table, we extract useful data to map protein families referred by Nguyen et. al.:

In [68]:
plf = features[['BRC ID', 'PATRIC genus-specific families (PLfams)']].astype("string")
plf.columns = ['BRC_ID', 'PLFam']
plf.set_index('BRC_ID', inplace = True)
plf

,PLFam
BRC_ID,
fig|272620.9.peg.923,PLF_570_00001646
fig|272620.9.peg.924,PLF_570_00000827
fig|272620.9.peg.925,PLF_570_00002014
fig|272620.9.peg.10,PLF_570_00001421
fig|272620.9.peg.926,PLF_570_00002244
...,...
fig|272620.9.peg.5264,PLF_570_00006439
fig|272620.9.peg.5266,PLF_570_00023239
fig|272620.9.peg.5267,PLF_570_00005744


## Specialty Genes

The table containing specialty genes relates several genomic features to a relevant property. Here the table is filtered by the antibiotic resistance property

In [69]:
AMR_genes = pd.DataFrame(pd.read_csv('specialty_genes.csv')['BRC ID'])
AMR_genes.columns = ['BRC_ID']

## Data from Nguyen et. al.

In [70]:
plf_500 = []

datadir = f'{root}/Nguyen_et_al_2020/{bacteria}/fasta.500.0'
for strain in listdir(datadir):
    with open(os.path.join(datadir, strain), 'r') as sequences:
        for line in sequences:
            if line[0] == '>':
                plf_500.append(line[1:len(line)-1])
                
plf_500 = pd.DataFrame(np.unique(plf_500))
plf_500.columns = ['Protein Family ID']

## RefSeq Mapping

In [71]:
refseq = features[['BRC ID', 'RefSeq Locus Tag']]
refseq.columns = ['BRC_ID', 'RefSeq']
refseq.set_index('BRC_ID', inplace = True)

In [72]:
AMR_refseq = features[['RefSeq Locus Tag']][features['BRC ID'].isin(AMR_genes['BRC_ID'])].reset_index(drop = True)
AMR_refseq.columns = ['AMR_RefSeq']

In [73]:
plf_map_refseq = features[['RefSeq Locus Tag', 'PATRIC genus-specific families (PLfams)']][features['PATRIC genus-specific families (PLfams)'].isin(plf_500['Protein Family ID'])].reset_index(drop = True)
plf_map_refseq.columns = ['RefSeq', 'PLF']
plf_map_refseq.dropna(inplace = True)
plf_map_refseq = plf_map_refseq.reset_index(drop = True)

In [74]:
plf_map_refseq

,RefSeq,PLF
0,KPN_00896,PLF_570_00000827
1,KPN_00900,PLF_570_00002125
2,KPN_00902,PLF_570_00000651
3,KPN_00909,PLF_570_00001323
4,KPN_00917,PLF_570_00001430
...,...,...
479,KPN_00081,PLF_570_00000905
480,KPN_00859,PLF_570_00002832
481,KPN_00868,PLF_570_00002022
482,KPN_00884,PLF_570_00000996


## Protein Interaction Network

In [75]:
ppi_patric = pd.read_csv('ppi_patric.csv')
ppi_patric = ppi_patric[['Interactor A ID', 'Interactor B ID']].astype("string")
ppi_patric.columns = ['Interactor_A_ID', 'Interactor_B_ID']
ppi_refseq = ppi_patric
for i in range(len(ppi_patric['Interactor_A_ID'])):
    ppi_refseq.at[i, 'Interactor_A_ID'] = refseq.loc[ppi_patric['Interactor_A_ID'][i]].RefSeq
        
for i in range(len(ppi_patric['Interactor_B_ID'])):
    ppi_refseq.at[i, 'Interactor_B_ID'] = refseq.loc[ppi_patric['Interactor_B_ID'][i]].RefSeq 

In [76]:
ppi_string = pd.read_csv('ppi_string.txt', sep = ' ')
ppi_string.columns = ['Interactor_A_ID', 'Interactor_B_ID', 'weight']
ppi_string.replace('272620.', '', regex = True, inplace = True)

In [86]:
ppi_all = pd.DataFrame(pd.concat([ppi_refseq, ppi_string], axis = 0)).reset_index()

In [87]:
ppi_all.fillna(999, inplace = True)

In [88]:
ppi_all

,index,Interactor_A_ID,Interactor_B_ID,weight
0,0,KPN_01979,KPN_03910,999.0
1,1,KPN_03878,KPN_04176,999.0
2,2,KPN_02823,KPN_03551,999.0
3,3,KPN_01375,KPN_02032,999.0
4,4,KPN_03530,KPN_03650,999.0
...,...,...,...,...
115027,110027,KPN_04868,KPN_04867,550.0
115028,110028,KPN_04867,KPN_04869,550.0
115029,110029,KPN_04869,KPN_04867,550.0
115030,110030,KPN_04868,KPN_04869,550.0


In [89]:
ppi = ppi_all[ppi_all['weight'] > 600].reset_index()

In [90]:
conserved_ppi_A = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_A_ID'])]['RefSeq']
conserved_ppi_B = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_B_ID'])]['RefSeq']

conserved_ppi = pd.DataFrame(pd.concat([conserved_ppi_A, conserved_ppi_B], axis = 0).drop_duplicates())

In [91]:
conserved_ppi

,RefSeq
0,KPN_00896
1,KPN_00900
2,KPN_00902
5,KPN_00923
7,KPN_00932
...,...
372,KPN_04222
377,KPN_04252
394,KPN_04433
417,KPN_04638


In [92]:
AMR_ppi_A = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_A_ID'])]['AMR_RefSeq']
AMR_ppi_B = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_B_ID'])]['AMR_RefSeq']

AMR_ppi = pd.DataFrame(pd.concat([AMR_ppi_A, AMR_ppi_B], axis = 0))
AMR_ppi.reset_index(drop=True, inplace=True)

In [93]:
AMR_ppi

,AMR_RefSeq
0,KPN_00911
1,KPN_00912
2,KPN_00096
3,KPN_01093
4,KPN_01284
...,...
117,KPN_04767
118,KPN_00806
119,KPN_00807
120,KPN_00808


In [94]:
ppi_info = pd.DataFrame(columns = ['Conserved Gene', 'Shortest Path to an AMR gene (length)',])

ppi_info['Conserved Gene'] = conserved_ppi.reset_index(drop = True)['RefSeq']

In [95]:
ppi_graph = nx.from_pandas_edgelist(ppi, 'Interactor_A_ID', 'Interactor_B_ID')

In [96]:
ppi_graph.number_of_edges()

20235

In [97]:
ppi_graph.number_of_nodes()

3099

In [98]:
idx = 0
for i in conserved_ppi['RefSeq']:
    lengths = []
    for j in AMR_ppi['AMR_RefSeq']:
        if nx.has_path(ppi_graph, i, j):
            lengths.append(nx.shortest_path_length(ppi_graph, i, j))
    if lengths:        
        ppi_info['Shortest Path to an AMR gene (length)'][idx] = min(lengths)
        
    idx += 1

In [99]:
print(ppi_info.groupby(['Shortest Path to an AMR gene (length)']).size().reset_index(name='Count'))

   Shortest Path to an AMR gene (length)  Count
0                                      1     56
1                                      2    165
2                                      3    102
3                                      4     12
4                                      5      2
5                                      6      2


In [100]:
plf_map_refseq = plf_map_refseq.set_index('RefSeq')

In [101]:
for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    gene_set = ppi_info[ppi_info['Shortest Path to an AMR gene (length)'] == i]['Conserved Gene']
    globals()[f'plf_length_{i}'] = plf_map_refseq.loc[list(gene_set)]['PLF']

In [102]:
bacdir = f'{root}/Fistarol_2022_2.0/{bacteria}'

os.mkdir(bacdir)

for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    newdir = f'length.{i}'
    lendir = os.path.join(bacdir, newdir)
    os.mkdir(path)
    
    sample = f'{root}/Nguyen_et_al_2020/{bacteria}/fasta.500.0'
    for strain in listdir(sample):
        with open(os.path.join(lendir, strain), 'a') as mystrain:
            with open(os.path.join(sample, strain), 'r') as sequences:
                first_loop = True
                for line in sequences:
                    if line[0] == '>':
                        if first_loop:
                            plfam = line[1:len(line)-1]
                            seq = ''
                            first_loop = False
                            continue
                        if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)
                        plfam = line[1:len(line)-1]
                        seq = ''
                    else:
                        seq += line
                if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)

At this point, it is possible to use this new configuration of data to run the model. We can take 25 genes to each strain for lengths equals to 1, 2 and 3, because the results from the paper also are derived from groups of 25 genes, hence, we can compare these results.

In [109]:
dir_25genes = os.path.join(bacdir, '25genes')
os.mkdir(dir_25genes)

for j in [0, 1, 2, 3, 4]:
    
    rand_idx = [sorted(random.sample(range(56), 25)), 
                sorted(random.sample(range(165), 25)), 
                sorted(random.sample(range(102), 25))]
    
    for i in range(1, len(rand_idx)+1): 
        
        lendir = os.path.join(bacdir, f'length.{i}')
        repdir = os.path.join(dir_25genes, f'length.{i}.{j}')
        os.mkdir(repdir)

        for strain in listdir(lendir):
            with open(os.path.join(repdir, strain), 'a') as mystrain:
                with open(os.path.join(lendir, strain), 'r') as sequences:
                    c = 0
                    first_loop = True
                    for line in sequences:
                        if line[0] == '>':
                            if first_loop:
                                plfam = line
                                seq = ''
                                c += 1
                                first_loop = False
                                continue
                            if c in rand_idx[i-1]:
                                mystrain.write(plfam)
                                mystrain.write(seq)
                            plfam = line
                            seq = ''
                            c += 1
                        else:
                            seq += line
                    if c in rand_idx[i-1]:
                            mystrain.write(plfam)
                            mystrain.write(seq)